# Loop 4 LB Feedback Analysis

**Submission Result**: CV 70.6224 | LB 70.6224 (gap: +0.0000)

**Key Insight**: CV-LB gap is ZERO! This means our local scoring is perfectly calibrated.

## Current Status
- Best LB: 70.6224
- Target: 68.8914
- Gap: 1.73 points (2.5%)

## Key Findings from Research

1. **jonathanchan kernel** shows top scores come from **ensembling 16+ external sources**:
   - GitHub: SmartManoj/Santa-Scoreboard
   - Telegram shared solutions
   - Multiple Kaggle datasets and notebooks
   
2. **The kernel uses a C++ optimizer (sa_v1_parallel)** with:
   - Fractional translation steps: [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
   - SA with perturb + local search
   - Population-based optimization (keeps top 3)

3. **Key strategy**: For each N, take the BEST solution from ALL sources

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import math
from numba import njit

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def calculate_scores_by_n(df):
    scores = {}
    for n in range(1, 201):
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        if len(group) != n:
            continue
        xs = strip(group['x'].values)
        ys = strip(group['y'].values)
        degs = strip(group['deg'].values)
        scores[n] = score_group(xs, ys, degs, TX, TY)
    return scores

print('Scoring functions ready')

Scoring functions ready


In [2]:
# Load our current best submission
df_current = pd.read_csv('/home/submission/submission.csv')
current_scores = calculate_scores_by_n(df_current)
current_total = sum(current_scores.values())
print(f'Current best score: {current_total:.6f}')
print(f'Target: 68.891380')
print(f'Gap: {current_total - 68.891380:.6f} ({(current_total - 68.891380) / 68.891380 * 100:.2f}%)')

Current best score: 70.622435
Target: 68.891380
Gap: 1.731055 (2.51%)


In [3]:
# Find all CSV files in snapshots that could be ensemble sources
import subprocess

# Search for all submission files in snapshots
result = subprocess.run(
    ['find', '/home/nonroot/snapshots', '-name', '*.csv', '-type', 'f'],
    capture_output=True, text=True, timeout=60
)

csv_files = [f for f in result.stdout.strip().split('\n') if f and 'submission' in f.lower()]
print(f'Found {len(csv_files)} potential submission files')
print('\nSample files:')
for f in csv_files[:20]:
    print(f'  {f}')

Found 2104 potential submission files

Sample files:
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/submission_best.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/submission_v18.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/submission_v21.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/002_preoptimized/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidates/candidate_000.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidates/candidate_004.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidates/candidate_002.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidat

In [4]:
# Score all potential ensemble sources
from tqdm import tqdm

ensemble_sources = []

for fp in tqdm(csv_files[:100], desc='Scoring files'):  # Limit to first 100 for speed
    try:
        df = pd.read_csv(fp)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            continue
        if len(df) < 20000:  # Should have ~20100 rows
            continue
        
        scores = calculate_scores_by_n(df)
        if len(scores) == 200:
            total = sum(scores.values())
            ensemble_sources.append({
                'file': fp,
                'total_score': total,
                'scores_by_n': scores
            })
    except Exception as e:
        continue

print(f'\nFound {len(ensemble_sources)} valid ensemble sources')
ensemble_sources.sort(key=lambda x: x['total_score'])

print('\nTop 10 sources by total score:')
for i, src in enumerate(ensemble_sources[:10]):
    print(f'{i+1}. {src["total_score"]:.6f} - {src["file"]}')

Scoring files:   0%|          | 0/100 [00:00<?, ?it/s]

Scoring files:   1%|          | 1/100 [00:00<00:47,  2.10it/s]

Scoring files:   2%|▏         | 2/100 [00:00<00:46,  2.12it/s]

Scoring files:   3%|▎         | 3/100 [00:01<00:45,  2.11it/s]

Scoring files:   4%|▍         | 4/100 [00:01<00:46,  2.08it/s]

...

Scoring files:  98%|█████████▊| 98/100 [00:45<00:00,  2.07it/s]

Scoring files:  99%|█████████▉| 99/100 [00:46<00:00,  2.09it/s]

Scoring files: 100%|██████████| 100/100 [00:46<00:00,  2.10it/s]

Scoring files: 100%|██████████| 100/100 [00:46<00:00,  2.15it/s]


Found 98 valid ensemble sources

Top 10 sources by total score:
1. 70.523320 - /home/nonroot/snapshots/santa-2025/21328309254/code/experiments/003_valid_ensemble/submission.csv
2. 70.615744 - /home/nonroot/snapshots/santa-2025/21328309254/code/submission2.csv
3. 70.615745 - /home/nonroot/snapshots/santa-2025/21328309254/code/experiments/003_bbox3_optimization/submission.csv
4. 70.615745 - /home/nonroot/snapshots/santa-2025/21328309254/code/submission_candidates/candidate_002.csv
5. 70.615745 - /home/nonroot/snapshots/santa-2025/21328309254/code/submission1.csv
6. 70.624381 - /home/nonroot/snapshots/santa-2025/21328309254/code/experiments/002_snapshot_ensemble/submission.csv
7. 70.624381 - /home/nonroot/snapshots/santa-2025/21328309254/code/submission_candidates/candidate_001.csv
8. 70.625918 - /home/nonroot/snapshots/santa-2025/21198893057/code/submission_candidates/candidate_026.csv
9. 70.625918 - /home/nonroot/snapshots/santa-2025/21198893057/code/submission_candidates/candidate_028

In [5]:
# Create ensemble: for each N, take the best solution from all sources
if len(ensemble_sources) > 0:
    best_per_n = {}
    best_source_per_n = {}
    
    for n in range(1, 201):
        best_score = float('inf')
        best_src = None
        
        for src in ensemble_sources:
            if n in src['scores_by_n']:
                if src['scores_by_n'][n] < best_score:
                    best_score = src['scores_by_n'][n]
                    best_src = src['file']
        
        best_per_n[n] = best_score
        best_source_per_n[n] = best_src
    
    ensemble_total = sum(best_per_n.values())
    print(f'Ensemble total score: {ensemble_total:.6f}')
    print(f'Current best: {current_total:.6f}')
    print(f'Improvement: {current_total - ensemble_total:.6f}')
    
    # Show which N values improved
    improved_n = []
    for n in range(1, 201):
        if n in current_scores and n in best_per_n:
            if best_per_n[n] < current_scores[n] - 1e-9:
                improved_n.append((n, current_scores[n] - best_per_n[n], best_source_per_n[n]))
    
    print(f'\nN values with improvement ({len(improved_n)} total):')
    improved_n.sort(key=lambda x: -x[1])  # Sort by improvement
    for n, imp, src in improved_n[:20]:
        print(f'  N={n:3d}: improvement={imp:.6f} from {src.split("/")[-1]}')

Ensemble total score: 70.523320
Current best: 70.622435
Improvement: 0.099115

N values with improvement (122 total):
  N=  5: improvement=0.022740 from submission.csv
  N=  2: improvement=0.013452 from submission.csv
  N= 56: improvement=0.011327 from submission.csv
  N= 55: improvement=0.008234 from submission.csv
  N=  4: improvement=0.005489 from submission.csv
  N= 54: improvement=0.004258 from submission.csv
  N= 71: improvement=0.003904 from submission.csv
  N= 53: improvement=0.003069 from submission.csv
  N= 65: improvement=0.002182 from submission.csv
  N= 80: improvement=0.001227 from submission.csv
  N= 47: improvement=0.001075 from submission.csv
  N=136: improvement=0.001008 from submission.csv
  N=140: improvement=0.000935 from submission.csv
  N=108: improvement=0.000931 from submission.csv
  N=168: improvement=0.000927 from submission.csv
  N= 77: improvement=0.000903 from submission.csv
  N= 69: improvement=0.000823 from submission.csv
  N=120: improvement=0.000797 fr

In [ ]:
# Build the ensemble submission
if len(ensemble_sources) > 0 and ensemble_total < current_total:
    print('Building ensemble submission...')
    
    rows = []
    for n in range(1, 201):
        best_src = best_source_per_n[n]
        if best_src:
            df_src = pd.read_csv(best_src)
            mask = df_src['id'].str.startswith(f'{n:03d}_')
            rows.append(df_src[mask][['id', 'x', 'y', 'deg']])
    
    df_ensemble = pd.concat(rows, ignore_index=True)
    
    # Sort properly
    df_ensemble['sn'] = df_ensemble['id'].str.split('_').str[0].astype(int)
    df_ensemble['si'] = df_ensemble['id'].str.split('_').str[1].astype(int)
    df_ensemble = df_ensemble.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
    
    # Verify score
    ensemble_scores = calculate_scores_by_n(df_ensemble)
    verified_total = sum(ensemble_scores.values())
    print(f'Verified ensemble score: {verified_total:.6f}')
    
    # Save
    os.makedirs('/home/code/experiments/005_ensemble', exist_ok=True)
    df_ensemble.to_csv('/home/code/experiments/005_ensemble/submission.csv', index=False)
    print('Saved to /home/code/experiments/005_ensemble/submission.csv')
else:
    print('No improvement from ensemble - all sources are worse or equal')